In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your zip file here


Saving ZIPPED_256x256_ct_grayscale.zip to ZIPPED_256x256_ct_grayscale.zip


In [ ]:
zip_filename = next(iter(uploaded))
print(f"✅ Uploaded file: {zip_filename}")


✅ Uploaded file: ZIPPED_256x256_ct_grayscale.zip


In [ ]:
import zipfile
import os

input_folder = "original_grayscale_256"
os.makedirs(input_folder, exist_ok=True)

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(input_folder)

print("✅ Extracted to folder:", input_folder)


✅ Extracted to folder: original_grayscale_256


In [ ]:
# Recursively find all images inside the extracted folder
image_files = []
for root, _, files in os.walk(input_folder):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_files.append(os.path.join(root, file))

print(f"📂 Number of extracted images (recursive): {len(image_files)} (expected: 125)")


📂 Number of extracted images (recursive): 125 (expected: 125)


In [ ]:
from PIL import Image, ImageOps

def get_all_transformations(image):
    transforms = []

    # Base transforms
    transforms.append(image)
    transforms.append(ImageOps.mirror(image))          # Horizontal flip
    transforms.append(ImageOps.flip(image))            # Vertical flip
    transforms.append(image.rotate(90))
    transforms.append(image.rotate(180))
    transforms.append(image.rotate(270))
    transforms.append(ImageOps.mirror(image.rotate(90)))
    transforms.append(ImageOps.mirror(image.rotate(180)))
    transforms.append(ImageOps.mirror(image.rotate(270)))
    transforms.append(ImageOps.flip(image.rotate(90)))
    transforms.append(ImageOps.flip(image.rotate(180)))
    transforms.append(ImageOps.flip(image.rotate(270)))
    transforms.append(image.transpose(Image.TRANSPOSE))
    transforms.append(image.transpose(Image.TRANSVERSE))

    # Repeat to reach 80 total
    while len(transforms) < 80:
        for t in transforms[:14]:
            transforms.append(t)
            if len(transforms) == 80:
                break

    return transforms


In [ ]:
test_img = Image.open(image_files[0]).convert("L").resize((256, 256))

test_augs = get_all_transformations(test_img)
print(f"🧪 Augmented versions generated: {len(test_augs)} (expected: 80)")


🧪 Augmented versions generated: 80 (expected: 80)


In [ ]:
output_folder = "augmented_grayscale_256"
os.makedirs(output_folder, exist_ok=True)

count = 0
image_index = 1

for path in image_files:
    img = Image.open(path).convert("L").resize((256, 256))

    augmented = get_all_transformations(img)

    for i, aug in enumerate(augmented):
        save_name = f"ct_{image_index:03d}_aug_{i+1:02d}.png"
        aug.save(os.path.join(output_folder, save_name))
        count += 1

    image_index += 1

print(f"✅ Augmentation done. Total saved images: {count}")


✅ Augmentation done. Total saved images: 10000


In [ ]:
from PIL import Image
sample_output = Image.open(os.path.join(output_folder, os.listdir(output_folder)[0]))
print("🧪 Sample image size:", sample_output.size)       # Should be (256, 256)
print("🧪 Sample image mode:", sample_output.mode)       # Should be 'L' (grayscale)


🧪 Sample image size: (256, 256)
🧪 Sample image mode: L


In [ ]:
zip_path = "augmented_grayscale_dataset.zip"
import zipfile

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for root, _, files in os.walk(output_folder):
        for file in files:
            filepath = os.path.join(root, file)
            arcname = os.path.relpath(filepath, output_folder)
            zipf.write(filepath, arcname)

print("✅ Zipped folder:", zip_path)


✅ Zipped folder: augmented_grayscale_dataset.zip


In [ ]:
print(f"🧪 Final zip size: {os.path.getsize(zip_path) / (1024 * 1024):.2f} MB")


🧪 Final zip size: 189.30 MB


In [ ]:
from google.colab import files as colab_files
colab_files.download(zip_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>